# CSC_52081_EP Project - Team 8

Advanced Machine Learning and Autonomous Agents Project

## Introduction

### Authors and Acknowledgmentss

This project was developed by in alphabetical order:

- Guilherme NUNES TROFINO:
    - [![LinkedIn](https://i.stack.imgur.com/gVE0j.png) LinkedIn](https://www.linkedin.com/in/guilherme-trofino/)
    - [![GitHub](https://i.stack.imgur.com/tskMh.png) GitHub](https://github.com/tr0fin0)

- Mateus GALVÃO:
    - [![LinkedIn](https://i.stack.imgur.com/gVE0j.png) LinkedIn](https://www.linkedin.com/in/galvao-mateus/)
    - [![GitHub](https://i.stack.imgur.com/tskMh.png) GitHub](https://github.com/Mateushga1)

- Rafael BENZI:
    - [![LinkedIn](https://i.stack.imgur.com/gVE0j.png) LinkedIn](https://www.linkedin.com/in/rafael-benzi/)
    - [![GitHub](https://i.stack.imgur.com/tskMh.png) GitHub](https://github.com/rafinhafpb)

We greatly appreciate our [CSC_52081_EP](https://synapses.polytechnique.fr/catalogue/2024-2025/ue/637/CSC-52081-EP-apprentissage-automatique-avance-et-agents-autonomes?from=D1) teachers at [École Polytechnique](https://www.polytechnique.edu/) in alphabetical order:

- Clément ELLIKER;
- Jérémie DECOCK;
- Jesse READ;
- Sonia VANIER;

## Installation

## Description